# Clean Architectures

## Use Case: Image Processing

In [1]:
from override_decorator import override
import time
from pathlib import Path

## Entities

In [2]:
from abc import ABC, abstractmethod
from typing import Any

class Entity(ABC):

    @abstractmethod
    def getDescription(self) -> str:
        raise NotImplementedError

## Interfaces / Actors

In [3]:
class Interface(ABC):

    @abstractmethod
    def execute(self, **kwargs) -> Entity:
        raise NotImplementedError

## Use Cases

In [4]:
class UseCase(ABC):

    @abstractmethod
    def execute(self, **kwargs) -> Entity:
        raise NotImplementedError

## Handlers

In [5]:
class Handler(ABC):

    def __init__(self) -> None:
        self._use_cases = {}

    def addUseCase(self, key: str, use_case: UseCase):
        self._use_cases[key] = use_case

    def removeUseCase(self, key: str):
        del self._use_cases[key]

    def getUseCase(self, key: str) -> UseCase:
        return self._use_cases[key]
    
    @abstractmethod
    def execute(self) -> Entity:
        raise NotImplementedError

# Implementation

## Entities

In [6]:
class Image(Entity):

    def __init__(self, image: Any) -> None:
        self._timestamp = self.setTimeStamp()
        self._image = image

    def setTimeStamp(self) -> None:
        timestamp = time.time()
        time_struct = time.localtime(timestamp) 
        timestamp = time.strftime("%Y-%m-%d %H:%M:%S", time_struct)
        self._timestamp = timestamp

    def setImage(self, image: Any) -> None:
        self._image = image

    def getTimeStamp(self) -> str:
        return self._timestamp
    
    def getImage(self):
        return self._image
    
    @override(Entity)
    def getDescription(self) -> str:
        return f"{self.__class__.__name__} (Entity)"

## Interface

In [7]:
class ImageReader(Interface):
        
    @abstractmethod
    def toImage(self, image: Any) -> Image:
        raise NotImplementedError
    
    @abstractmethod
    def readImage(self, image_path: Path) -> Any:
        raise NotImplementedError

In [8]:
from typing import Any
import cv2

class OpenCVImageReader(ImageReader):

    def __init__(self) -> None:
        print(f"{self.__class__.__name__}")

    @override(ImageReader)
    def toImage(self, image: Any) -> Image:
        return Image(image)

    @override(ImageReader)
    def readImage(self, image_path: Path) -> Any:
        return cv2.imread(str(image_path))
    
    @override(Interface)
    def execute(self, **kwargs) -> Entity:
        
        image = self.readImage(kwargs["image_path"])
        image = self.toImage(image)

        return image

## Use Case

In [ ]:
class ReadImageUseCase(UseCase):

    def __init__(self, image_reader: ImageReader):
        self._image_reader = image_reader

    @override(UseCase)
    def execute(self, image_path: str) -> Entity:
        return self._image_reader.execute(image_path=image_path)